---
title: GCP cloud cost simulator widget
---

Enable computation on this page ⏻, then execute it ►, to display an interactive calculator. It can show you the cost for running CPU heavy workloads in standard virtual machines versus CPU optimized machines.

```{code-cell} python
:tags: remove-input
import sys

# Check if running inside JupyterLite (Pyodide)
if "pyodide" in sys.modules:
    import micropip
    # Install ipywidgets in JupyterLite
    await micropip.install("ipywidgets")
```

```{code-cell} python
:tags: remove-input
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout
```

```{code-cell} python
:tags: remove-input
# Pricing data for the latest instance costs in the Iowa Central region (us-central1)
NODE_PRICING = {
    'n2-highcpu-32': 27.93,  # Latest price for n2-highcpu-32
    'n2-highcpu-96': 83.73,  # Latest price for n2-highcpu-96
    'n2-highmem-32': 50.31,  # Latest price for n2-highmem-32
}
```

```{code-cell} python
:tags: remove-input
# Function to calculate the total number of nodes and total cost
def calculate_total_cost(node_type, num_cpus, num_students, num_days):
    # Extract the number of CPUs in the node from the node type (e.g., 32 or 96)
    node_capacity = int(node_type.split('-')[-1])
    
    # Calculate the number of students allocated per node
    students_per_node = node_capacity / num_cpus
    
    # Calculate the total number of nodes needed
    total_nodes = num_students / students_per_node
    
    # Get the node cost from the pricing dictionary
    node_cost = NODE_PRICING[node_type]
    
    # Calculate the total cost for the given number of days
    total_cost = total_nodes * node_cost * num_days
    
    return total_nodes, total_cost

# Callback function for the button
def on_button_click(b):
    node_type = node_type_dropdown.value
    num_cpus = int(num_cpus_dropdown.value)  # Extracting the number of CPUs (2 or 4)
    num_students = int(students_input.value)
    num_days = int(num_days_input.value)
    
    # Calculate total nodes and total cost
    total_nodes, total_cost = calculate_total_cost(node_type, num_cpus, num_students, num_days)
    
    # Show result in text box
    result_text.value = f"Total nodes allocated: {total_nodes:.2f}\nTotal cost for {num_days} days: ${total_cost:.2f}"
```

```{code-cell} python
:tags: remove-input
# Widget elements
node_type_dropdown = widgets.Dropdown(
    options=['n2-highcpu-32', 'n2-highcpu-96', 'n2-highmem-32'],
    value='n2-highcpu-32',
    description='Node Type:',
    style={'description_width': 'auto'},
    layout=Layout(width='50%', padding='1em')
)

num_cpus_dropdown = widgets.Dropdown(
    options=['2', '4'],
    value='4',
    description='CPUs per Student:',
    style={'description_width': 'auto'},
    layout=Layout(width='50%', padding='1em')
)

students_input = widgets.IntText(
    value=1200,
    description='Total Students:',
    style={'description_width': 'auto'},
    layout=Layout(width='50%', padding='1em')
)

num_days_input = widgets.IntText(
    value=12,
    description='Number of Days:',
    style={'description_width': 'auto'},
    layout=Layout(width='50%', padding='1em')
)
```

```{code-cell} python
:tags: remove-input
calculate_button = widgets.Button(
    description='Calculate Total Cost',
    button_style='success',
    layout=Layout(width='50%')  # Set button width to 50% of the container width
)

# Improved result text box with larger size, center alignment, and padding for better spacing
result_text = widgets.Textarea(
    value='',
    description='Result:',
    disabled=True,
    layout=Layout(width='50%', height='4em', padding='1em'),
    style={'description_width': 'initial', 'text-align': 'center'}
)

# Attach the click event handler
calculate_button.on_click(on_button_click)

# Organize the widgets in two columns (HBox)
input_columns = widgets.VBox([node_type_dropdown, num_cpus_dropdown, students_input, num_days_input, calculate_button], layout=Layout(width='100%', padding='1em'))

# Final layout
final_layout = widgets.VBox([
    input_columns,
    #calculate_button,
    result_text
], layout=Layout(padding='1em'))
```

```{code-cell} python
:tags: remove-input
# Display the entire widget setup
display(final_layout)
```